<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys
from IPython.display import display, HTML, Markdown
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from tf.extra.cunei import Cunei

In [3]:
A = Cunei('~/github', 'Nino-cunei/uruk', 'calculator')
A.api.makeAvailableIn(globals())

Found 2095 ideograph linearts
Found 2724 tablet linearts
Found 5495 tablet photos


**Documentation:** <a target="_blank" href="https://github.com/Nino-cunei/uruk/blob/master/docs/about.md" title="provenance of this corpus">Uruk IV-III (v1.0)</a> <a target="_blank" href="https://github.com/Nino-cunei/uruk/blob/master/docs/transcription.md" title="feature documentation">Feature docs</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/Cunei/" title="cunei api documentation">Cunei API</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/" title="text-fabric-api">Text-Fabric API 5.0.4</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/#search-templates" title="Search Templates Introduction and Reference">Search help</a>


This notebook online:
<a target="_blank" href="http://nbviewer.jupyter.org/github/Nino-cunei/tutorials/blob/master/bits-and-pieces/calculator.ipynb">NBViewer</a>
<a target="_blank" href="https://github.com/Nino-cunei/tutorials/blob/master/bits-and-pieces/calculator.ipynb">GitHub</a>


In [4]:
C.levels.data

(('tablet', 22.013513513513512, 143889, 150252),
 ('face', 14.098032994923859, 150253, 159708),
 ('column', 9.338016116380233, 180450, 194472),
 ('line', 3.6104012052898833, 227226, 263067),
 ('case', 3.4648222982074395, 159709, 169359),
 ('cluster', 1.031416969437914, 194473, 227225),
 ('quad', 2.050606220347918, 140095, 143888),
 ('comment', 1.0, 169360, 180449),
 ('sign', 1, 1, 140094))

In [5]:
shinPP = dict(
    N41=0.2,
    N04=1,
    N19=6,
    N46=60,
    N36=180,
    N49=1800,
)

shinPPPat = '|'.join(shinPP)

We query for shinPP numerals on the faces of tablets.
The result of the query is a list of tuples `(t, f, s)` consisting of
a tablet node, a face node and a sign node, which is a shinPP numeral.

In [13]:
def condense(results):
    otype = F.otype.v
    up = L.u
    tablets = {}
    for ns in results:
      for n in ns:
        if otype(n) == 'tablet':
          tablets.setdefault(n, set())
        else:
          t = up(n, otype='tablet')[0]
          tablets.setdefault(t, set()).add(n)
    return tuple((t, ) + tuple(tablets[t]) for t in sortNodes(tablets))

In [14]:
query = f'''
tablet
    sign type=numeral
'''
results = A.search(query)

  0.39s 38122 results


In [15]:
cresults = condense(results)

In [16]:
len(cresults)

5071

In [18]:
cresults[0:10]

((143889, 5),
 (143890, 26, 20, 28, 22),
 (143891, 40, 41, 39),
 (143892, 64, 67, 68, 69, 70, 77, 56, 59, 60, 61),
 (143893, 98, 101, 88, 91, 94),
 (143894, 135, 108, 110, 142, 143, 119, 120, 122),
 (143895, 152, 148, 151),
 (143896, 161, 164, 166, 170, 172, 173, 176, 177, 179),
 (143897, 195, 197, 201, 204, 205, 206, 209, 211, 214, 215, 218, 187, 190),
 (143898, 232, 226, 229, 223))

In [5]:
query = f'''
tablet
    face
        sign type=numeral grapheme={shinPPPat}
'''

In [6]:
results = list(S.search(query))
len(results)

1018

In [7]:
numerals = {}
pNums = {}
for (tablet, face, sign) in results:
    pNums[F.catalogId.v(tablet)] = tablet
    numerals.setdefault(tablet, {}).setdefault(face, []).append(sign)
len(pNums)

235

We show the tablets, the shinPP numerals per face, and we add up the numerals per face.

In [8]:
def dm(x): display(Markdown(x))

def showResult(pNum, tabletLineart):
    dm('---\n')
    tablet = pNums.get(pNum, None)
    if tablet is None:
        dm(f'**no results for {pNum}**')
    if tabletLineart:
        display(A.lineart(tablet, withCaption="top", width="200"))
    faces = numerals[tablet]
    for (face, signs) in faces.items():
        dm(f'### {F.type.v(face)}')
        distinctSigns = {}
        for s in signs:
            distinctSigns.setdefault(A.atfFromSign(s), []).append(s)
        display(A.lineart(distinctSigns))
        total = 0
        for (signAtf, signs) in distinctSigns.items():
            # note that all signs for the same signAtf have the same grapheme and repeat
            value = 0
            for s in signs:
                value += F.repeat.v(s) * shinPP[F.grapheme.v(s)]
            total += value
            amount = len(signs)
            shinPPval = shinPP[F.grapheme.v(signs[0])]
            repeat = F.repeat.v(signs[0])
            print(f'{amount} x {signAtf} = {amount} x {repeat} x {shinPPval} = {value}')
        dm(f'**total** = **{total}**')

Below you can select any of the tablets with ShinPP numerals on it, and see how its faces add up.

In [9]:
widget = interactive(showResult, pNum=sorted(pNums), tabletLineart=False)
display(widget)

---


### obverse

None

1 x 1(N04) = 1 x 1 x 1 = 1


**total** = **1**